In [2]:
import sys
import os
import os.path as osp

import numpy as np
import cv2
import matplotlib.pyplot as plt
import csv
import argparse

import torch
from torch.utils import data
import torch.nn.functional as F
from torch.utils.data import DataLoader

# sys.path.insert(1, os.path.join(sys.path[0], '..'))
import datasets
from datasets import build_dataset
import util.misc as utils

In [3]:
def get_args_parser():
    parser = argparse.ArgumentParser('Set transformer detector', add_help=False)
    parser.add_argument('--lr', default=1e-4, type=float)
    parser.add_argument('--lr_backbone', default=1e-5, type=float)
    parser.add_argument('--batch_size', default=100, type=int)
    parser.add_argument('--weight_decay', default=1e-4, type=float)
    parser.add_argument('--epochs', default=50, type=int)
    parser.add_argument('--lr_drop', default=20, type=int)
    parser.add_argument('--clip_max_norm', default=0.1, type=float,
                        help='gradient clipping max norm')
    
    # * Backbone
    parser.add_argument('--backbone', default='resnet50', type=str,
                        help="Name of the convolutional backbone to use")
    parser.add_argument('--dilation', action='store_true',
                        help="If true, we replace stride with dilation in the last convolutional block (DC5)")
    parser.add_argument('--position_embedding', default='sine', type=str, choices=('sine', 'learned'),
                        help="Type of positional embedding to use on top of the image features")

    # * Transformer
    parser.add_argument('--enc_layers', default=6, type=int,
                        help="Number of encoding layers in the transformer")
    parser.add_argument('--dec_layers', default=6, type=int,
                        help="Number of decoding layers in the transformer")
    parser.add_argument('--dim_feedforward', default=2048, type=int,
                        help="Intermediate size of the feedforward layers in the transformer blocks")
    parser.add_argument('--hidden_dim', default=256, type=int,
                        help="Size of the embeddings (dimension of the transformer)")
    parser.add_argument('--dropout', default=0.1, type=float,
                        help="Dropout applied in the transformer")
    parser.add_argument('--nheads', default=8, type=int,
                        help="Number of attention heads inside the transformer's attentions")
    parser.add_argument('--num_queries', default=3, type=int,
                        help="Number of query slots")
    parser.add_argument('--pre_norm', action='store_true')
    
    # * Segmentation
    parser.add_argument('--masks', action='store_true',
                        help="Train segmentation head if the flag is provided")

    # Loss
    parser.add_argument('--no_aux_loss', dest='aux_loss', action='store_false',
                        help="Disables auxiliary decoding losses (loss at each layer)")
    
    # dataset parameters
    parser.add_argument('--input_width', default=512, type=int)
    parser.add_argument('--input_height', default=512, type=int)    
    
    parser.add_argument('--output_dir', default='output',
                        help='path where to save, empty for no saving')
    parser.add_argument('--device', default='cuda',
                        help='device to use for training / testing')

    # display
    parser.add_argument('--display', default=True,
                        help='choose whether visualize image')
    return parser

args = get_args_parser().parse_args(args=[])

In [4]:
dataset_test = build_dataset(image_set='test', args=args)
sampler_test = torch.utils.data.SequentialSampler(dataset_test)
data_loader_test = DataLoader(dataset_test, 1, sampler=sampler_test,
                             drop_last=False, 
                             collate_fn=utils.collate_fn, 
                             num_workers=2)

In [12]:
img, target = dataset_test[128]
print(target.keys())

dict_keys(['rp', 'fovy', 'up_vector', 'focal', 'zvp', 'hl', 'org_img', 'org_sz', 'input_sz', 'img_path', 'filename'])


In [50]:
focal = target['focal']
zvp = target['zvp']
hl = target['hl']
print(focal, focal.size())
print(zvp, zvp.size())
print(hl, hl.size())

tensor([1.4281]) torch.Size([1])
tensor([1.7051, 5.9464, 1.0000]) torch.Size([3])
tensor([-0.2686, -0.9366, -0.3213]) torch.Size([3])


In [51]:
zvp[1]*(-hl[2]/hl[1])

tensor(-2.0396)

In [52]:
focal*focal

tensor([2.0396])

In [53]:
print(zvp*hl)
print(torch.sum(zvp*hl))

tensor([-0.4579, -5.5695, -0.3213])
tensor(-6.3487)


In [56]:
zvp_focal = zvp.clone()
zvp_focal[2] = focal[0]
print(zvp_focal)

hl_focal = hl.clone()
hl_focal[0:2] *= focal[0]
print(hl_focal)

tensor([1.7051, 5.9464, 1.4281])
tensor([-0.3836, -1.3376, -0.3213])


In [60]:
print(zvp_focal*hl_focal)
cossim = F.cosine_similarity(zvp_focal, hl_focal, dim=-1).abs()
print(1.0 - cossim)

tensor([-0.6540, -7.9541, -0.4588])
tensor(0.)


In [7]:
dic = {'c': 0, 'd': 0}
print(type(dic.keys()))
lst = ['a','b'] + list(dic.keys())
print(lst)

<class 'dict_keys'>
['a', 'b', 'c', 'd']
